In [1]:
import csv
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url_xls = "https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide"
url_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"

In [3]:
page = requests.get(url_xls).text

In [4]:
soup = BeautifulSoup(page, 'html.parser')

In [5]:
data_attribute = soup.find_all("a", attrs={"data-toggle": "tooltip"})

In [6]:
for link in data_attribute:
    data_link = link.get('href')

In [7]:
with requests.Session() as s:
    download = s.get(data_link).content
    dataframe = pd.read_excel(download)

In [8]:
dataframe.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
0,2020-03-21,21,3,2020,2,0,Afghanistan,AF
1,2020-03-20,20,3,2020,0,0,Afghanistan,AF
2,2020-03-19,19,3,2020,0,0,Afghanistan,AF
3,2020-03-18,18,3,2020,1,0,Afghanistan,AF
4,2020-03-17,17,3,2020,5,0,Afghanistan,AF


In [9]:
dataframe.groupby("Countries and territories").mean().head()

,Day,Month,Year,Cases,Deaths
Countries and territories,,,,,
Afghanistan,15.416667,1.861111,2019.986111,0.333333,0.000000
Albania,15.000000,3.000000,2020.000000,5.384615,0.153846
Algeria,14.987013,1.935065,2019.987013,1.220779,0.129870
Andorra,16.000000,3.000000,2020.000000,9.375000,0.000000
Antigua_and_Barbuda,18.000000,3.000000,2020.000000,0.500000,0.000000


In [11]:
df_github = pd.read_csv(url_csv)

In [12]:
data_by_country = df_github.groupby("Country/Region").mean()

In [13]:
df_github.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,59,70,75,82,114,147,177,212,272,322
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,639,639,701,773,839,825,878,889,924,963
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,178,178,200,212,226,243,266,313,345,385
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,149,149,197,238,428,566,673,790,900,1030


In [28]:
df_github_countries = df_github.iloc[:, 0:4]

In [29]:
df_github_countries

,Province/State,Country/Region,Lat,Long
0,NaN,Thailand,15.0000,101.0000
1,NaN,Japan,36.0000,138.0000
2,NaN,Singapore,1.2833,103.8333
3,NaN,Nepal,28.1667,84.2500
4,NaN,Malaysia,2.5000,112.5000
...,...,...,...,...
472,Sint Maarten,Netherlands,18.0425,-63.0548
473,NaN,Niger,17.6078,8.0817
474,NaN,Papua New Guinea,-6.3150,143.9555
475,Isle of Man,United Kingdom,54.2361,-4.5481


In [119]:
df_github_countries.iloc[0,0]

nan

In [62]:
country = df_github_countries.iloc[:, 1:4].drop_duplicates(subset=['Country/Region'],keep='last',ignore_index=True)

In [63]:
len(country)

162

In [65]:
country.to_csv("country.csv")

In [66]:
continents = ["Africa", "Antarctica", "Asia", "Australia", "Europe", "North America", "South America"]

In [67]:
continents_df = pd.DataFrame(continents)

In [69]:
continents_df.to_csv("continent.csv")

In [93]:
prov_state = df_github.iloc[:, 0].drop_duplicates(keep='last')

In [94]:
prov_state = prov_state.reset_index()

In [95]:
prov_state = prov_state.drop(["index"],axis=1)

In [110]:
prov_state.iloc[322, :] = "No province/state"

In [146]:
provstate_noquotes = []
for provstate in prov_state.iloc[:,0]:
    provstate_noquotes.append(provstate.replace('"', ''))

In [147]:
prov_state['Province/State_noquotes'] = provstate_noquotes

In [144]:
prov_state=prov_state.drop("Province/State",axis=1)

In [151]:
prov_state.to_csv("province_state.csv")

In [128]:
import math
region_codes = []
for region in df_github_countries.iloc[:,0]:
    if isinstance(region, str):
        pass
    elif np.isnan(region):
        region_codes.append(322)
        continue
    for i, item in enumerate(prov_state.iloc[:,0]):
        if item == region:
            region_codes.append(i)

print(len(region_codes))

477


In [130]:
df_github_countries["region_codes"] = region_codes

In [131]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes
0,NaN,Thailand,15.0000,101.0000,322
1,NaN,Japan,36.0000,138.0000,322
2,NaN,Singapore,1.2833,103.8333,322
3,NaN,Nepal,28.1667,84.2500,322
4,NaN,Malaysia,2.5000,112.5000,322
...,...,...,...,...,...
472,Sint Maarten,Netherlands,18.0425,-63.0548,320
473,NaN,Niger,17.6078,8.0817,322
474,NaN,Papua New Guinea,-6.3150,143.9555,322
475,Isle of Man,United Kingdom,54.2361,-4.5481,321


In [156]:
asia_list = ["Thailand", "Japan", "Singapore"]
australia_list = ["Australia"]
continent_codes = []
for item in df_github_countries.iloc[:,1]:
    if item in asia_list:
        continent_codes.append(2)
    else:
        continent_codes.append("XXX")

In [157]:
continent_codes

[2,
 2,
 2,
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XXX',
 'XX